In [110]:
import pandas as pd
import numpy as np
import re
import os

In [111]:
df = pd.read_csv('../data/preprocessed/full.csv')

## Remove Duplicates

In [112]:
df.head(5)

,address,area,city_province,district,front_road_width,interior,latitude,legal,longitude,n_bathrooms,n_bedrooms,n_floors,price,property_type,source
0,"Dự án Masteri Thảo Điền, Đường Xa Lộ Hà Nội, P...",75.0,Hồ Chí Minh,Quận 2,NaN,Đầy đủ,10.802441,Sổ đỏ/ Sổ hồng,106.741950,NaN,2.0,NaN,7.7,Căn hộ chung cư,batdongsancomvn
1,"Phúc Lợi, Phúc Lợi, Long Biên, Hà Nội",72.0,Hà Nội,Long Biên,6.10,Đầy đủ,21.038832,Sổ đỏ/ Sổ hồng,105.934539,6.0,6.0,6.0,15.8,Nhà riêng,batdongsancomvn
2,"Phường 12, Đà Lạt, Lâm Đồng",102.3,Lâm Đồng,Đà Lạt,5.00,Đầy đủ,11.973893,Sổ đỏ/ Sổ hồng,108.484595,1.0,2.0,2.0,7.5,Nhà riêng,batdongsancomvn
3,"Đường Phạm Văn Chiêu, Phường 9, Gò Vấp, Hồ Chí...",60.9,Hồ Chí Minh,Gò Vấp,4.35,Cơ bản,10.849654,Sổ đỏ/ Sổ hồng,106.651222,2.0,2.0,1.0,6.6,Nhà riêng,batdongsancomvn
4,"Đường Đồng Me, Phường Mễ Trì, Nam Từ Liêm, Hà Nội",148.0,Hà Nội,Nam Từ Liêm,6.50,Đầy đủ,21.011918,Sổ đỏ/ Sổ hồng,105.776275,41.0,41.0,10.0,59.0,Nhà riêng,batdongsancomvn


## Standardization

1. **property_type**

In [68]:
print(f"Dataset shape before: {df.shape}")
print("Unique property_type (raw):")
print(df["property_type"].value_counts(dropna = False))

PROPERTY_TYPE_MAP = {
    "Nhà riêng": [
        "nhà mặt tiền",
        "nhà mặt phố",
        "mặt tiền",
        "nhà hẻm ngõ",
        "nhà hẻm, ngõ",
        "nhà riêng"
    ],
    "Biệt thự": [
        "biệt thự",
        "biệt thự, villa",
        "nhà biệt thự, liền kề"
    ],
    "Chung cư": [
        "căn hộ chung cư",
        "căn hộ chung cư mini"
    ],
    "Shophouse": [
        "shophouse, nhà phố thương mại"
    ],
    "Condotel": [
        "condotel"
    ]
}

def normalize_property_type(value):
    if pd.isna(value):
        return pd.NA

    value_norm = str(value).strip().lower()

    for standard_type, variants in PROPERTY_TYPE_MAP.items():
        for v in variants:
            if v.lower() in value_norm:
                return standard_type

    return "Khác"

df["property_type"] = df["property_type"].apply(normalize_property_type)

print("\nUnique property_type (normalized):")
print(df["property_type"].value_counts(dropna = False))

Dataset shape before: (22245, 15)
Unique property_type (raw):
property_type
Nhà riêng                                           4378
Căn hộ chung cư                                     4318
Nhà mặt phố                                         4152
Nhà biệt thự, liền kề                               3980
Nhà hẻm ngõ                                         2344
Nhà mặt tiền                                        2084
Mặt tiền                                             512
Biệt thự                                             469
Nhà hẻm, ngõ                                           4
=======                                                1
property_type                                          1
>>>>>>> 84ea7898394093291f73460797a0c40d3bc59366       1
Biệt thự, Villa                                        1
Name: count, dtype: int64

Unique property_type (normalized):
property_type
Nhà riêng    13474
Biệt thự      4450
Chung cư      4318
Khác             3
Name: count, dtype: int64


In [69]:
df = df[df["property_type"] != "Khác"].reset_index(drop = True)
print(df["property_type"].value_counts(dropna = False))

property_type
Nhà riêng    13474
Biệt thự      4450
Chung cư      4318
Name: count, dtype: int64


2. **legal**

In [70]:
import unicodedata

def normalize_text(text):
    if pd.isna(text):
        return np.nan
    text = str(text).lower().strip()
    text = unicodedata.normalize("NFKD", text)
    text = "".join(c for c in text if not unicodedata.combining(c))
    text = re.sub(r"\s+", " ", text)
    return text


In [71]:
def normalize_legal(val):
    if pd.isna(val):
        return "Không có"

    v = normalize_text(val)

    # --- GROUP 1: SỔ RIÊNG / CHÍNH CHỦ ---
    if any(k in v for k in [
        "so do", "so hong", "so rieng", "chinh chu",
        "bia do", "shr", "tho cu", "hoan cong",
        "vinh vien", "lau dai"
    ]):
        return "Sổ riêng"

    # --- GROUP 2: SỔ CHUNG / ĐỒNG SỞ HỮU ---
    if any(k in v for k in [
        "so chung", "dong so huu", "su dung chung",
        "so do chung", "so hong chung", "dsh"
    ]):
        return "Sổ chung"

    # --- GROUP 3: HỢP ĐỒNG / CHỜ SỔ ---
    if any(k in v for k in [
        "hop dong", "hdmb", "dang cho so", "cho so",
        "chu dau tu", "50 nam", "70 nam", "tmdv"
    ]):
        return "Hợp đồng"

    # --- GROUP 4: VI BẰNG / THOẢ THUẬN ---
    if any(k in v for k in [
        "vi bang", "mua ban vi bang",
        "van ban thoa thuan", "vbtt"
    ]):
        return "Vi bằng"

    # --- GROUP 5: KHÔNG RÕ ---
    return "Không có"


df["legal"] = df["legal"].apply(normalize_legal)

print(df["legal"].value_counts(dropna = False))

legal
Sổ riêng    13938
Không có     8102
Hợp đồng      188
Sổ chung        8
Vi bằng         6
Name: count, dtype: int64


3. **interior**

In [72]:
df["interior"].value_counts(dropna = False)

interior
NaN                                                 11892
Đầy đủ                                               4744
Cơ bản                                               2250
Không nội thất                                        970
Đầy đủ.                                               692
                                                    ...  
Bàn giao hoàn thiện cơ bản                              2
Nội thất đầy đủ như điều hòa, tủ lạnh, giường...        2
Chủ để lại toàn bộ nội thất.                            2
Nội thất full nội thất cơ bản                           2
Nội thất gần ful.                                       2
Name: count, Length: 340, dtype: int64

In [73]:
if "interior" in df.columns:

    def normalize_interior(val):
        if pd.isna(val):
            return np.nan

        val = str(val).lower().strip()

        # không nội thất
        if re.search(r"không|trống|thô|chưa", val):
            return "Không nội thất"

        # cao cấp
        if re.search(r"cao cấp|luxury|sang|xịn|nhập khẩu", val):
            return "Nội thất cao cấp"

        # full nội thất
        if re.search(r"full|đầy đủ", val):
            return "Nội thất đầy đủ"

        # nội thất cơ bản
        if re.search(r"cơ bản|basic|có sẵn", val):
            return "Nội thất cơ bản"

        return np.nan

    df["interior"] = df["interior"].apply(normalize_interior)

In [74]:
df["interior"].value_counts(dropna = False)

interior
NaN                 12096
Nội thất đầy đủ      5924
Nội thất cơ bản      2622
Không nội thất       1234
Nội thất cao cấp      366
Name: count, dtype: int64

In [75]:
# Flag for having/not having interior
df["has_interior_info"] = df["interior"].notna().astype(int)
print(df["has_interior_info"].value_counts())

has_interior_info
0    12096
1    10146
Name: count, dtype: int64


## Remove Outliers

In [76]:
print("Initial number of records: " + str(df.shape[0]))

Initial number of records: 22242


1. **price**

In [77]:
df = df[df["price"] >= 0.5]

df = df[~(
    (df["property_type"] == "Nhà riêng") &
    (df["price"] > 500)
)]

print("Number of records: " + str(df.shape[0]))

Number of records: 20967


2. **area**

In [78]:
df = df[(df["area"] >= 10) & (df["area"] <= 1500)]
print("Number of records: " + str(df.shape[0]))

Number of records: 20747


3. **price_per_m2**

In [79]:
df["price_per_m2"] = (df["price"] * 1000) / df["area"]

df = df[
    (df["price_per_m2"] >= 5) &
    (df["price_per_m2"] <= 2000)
]
print("Number of records: " + str(df.shape[0]))

Number of records: 20583


## Deal with Missing Data

1. **longtitude** and **latitude**

In [80]:
print("Number of missing values on latitude: " + str(df['latitude'].isna().sum()))
print("Number of missing values on longitude: " + str(df['longitude'].isna().sum()))

Number of missing values on latitude: 260
Number of missing values on longitude: 260


In [81]:
df["latitude"] = df["latitude"].fillna(
    df.groupby(["city_province", "district"])["latitude"].transform("mean")
)

df["longitude"] = df["longitude"].fillna(
    df.groupby(["city_province", "district"])["longitude"].transform("mean")
)

print("Number of missing values on latitude: " + str(df['latitude'].isna().sum()))
print("Number of missing values on longitude: " + str(df['longitude'].isna().sum()))

Number of missing values on latitude: 258
Number of missing values on longitude: 258


In [82]:
df["latitude"] = df["latitude"].fillna(
    df.groupby("city_province")["latitude"].transform("mean")
)

df["longitude"] = df["longitude"].fillna(
    df.groupby("city_province")["longitude"].transform("mean")
)

print("Number of missing values on latitude: " + str(df['latitude'].isna().sum()))
print("Number of missing values on longitude: " + str(df['longitude'].isna().sum()))

Number of missing values on latitude: 7
Number of missing values on longitude: 7


In [83]:
df = df.dropna(subset = ["latitude", "longitude"])

print("Number of missing values on latitude: " + str(df['latitude'].isna().sum()))
print("Number of missing values on longitude: " + str(df['longitude'].isna().sum()))

Number of missing values on latitude: 0
Number of missing values on longitude: 0


2. **n_floors**, **n_bedrooms** and **n_bathrooms**

In [84]:
print("Number of missing values on n_floors: " + str(df['n_floors'].isna().sum()))
print("Number of missing values on n_bedroomss: " + str(df['n_bedrooms'].isna().sum()))
print("Number of missing values on n_bathrooms: " + str(df['n_bathrooms'].isna().sum()))

Number of missing values on n_floors: 5634
Number of missing values on n_bedroomss: 3526
Number of missing values on n_bathrooms: 4116


In [85]:
group_cols = ["property_type", "district", "city_province"]
num_cols = ["n_floors", "n_bedrooms", "n_bathrooms"]

for col in num_cols:
    df[col] = df[col].fillna(
        df.groupby(group_cols)[col].transform("median")
    )

print("Number of missing values on n_floors: " + str(df['n_floors'].isna().sum()))
print("Number of missing values on n_bedroomss: " + str(df['n_bedrooms'].isna().sum()))
print("Number of missing values on n_bathrooms: " + str(df['n_bathrooms'].isna().sum()))

Number of missing values on n_floors: 3758
Number of missing values on n_bedroomss: 60
Number of missing values on n_bathrooms: 82


Fallback to avoid that median = NaN

In [86]:
for col in num_cols:
    df[col] = df[col].fillna(
        df.groupby(["property_type", "city_province"])[col]
          .transform("median")
    )

print("Number of missing values on n_floors: " + str(df['n_floors'].isna().sum()))
print("Number of missing values on n_bedroomss: " + str(df['n_bedrooms'].isna().sum()))
print("Number of missing values on n_bathrooms: " + str(df['n_bathrooms'].isna().sum()))

Number of missing values on n_floors: 814
Number of missing values on n_bedroomss: 10
Number of missing values on n_bathrooms: 8


Rule-based correction

In [87]:
# Chung cư commonly has 1 floors
df.loc[
    (df["property_type"] == "Chung cư") & (df["n_floors"].isna()),
    "n_floors"
] = 1

# Nhà riêng has at least 1 floors
df.loc[
    (df["property_type"] == "Nhà riêng") & (df["n_floors"].isna()),
    "n_floors"
] = 1

# Biệt thự commonly has >= 3 bedrooms
df.loc[
    (df["property_type"] == "Biệt thự") & (df["n_bedrooms"].isna()),
    "n_bedrooms"
] = 3

print("Number of missing values on n_floors: " + str(df['n_floors'].isna().sum()))
print("Number of missing values on n_bedroomss: " + str(df['n_bedrooms'].isna().sum()))
print("Number of missing values on n_bathrooms: " + str(df['n_bathrooms'].isna().sum()))

Number of missing values on n_floors: 2
Number of missing values on n_bedroomss: 8
Number of missing values on n_bathrooms: 8


In [88]:
df = df.dropna(subset = num_cols)

print("Number of missing values on n_floors: " + str(df['n_floors'].isna().sum()))
print("Number of missing values on n_bedroomss: " + str(df['n_bedrooms'].isna().sum()))
print("Number of missing values on n_bathrooms: " + str(df['n_bathrooms'].isna().sum()))

Number of missing values on n_floors: 0
Number of missing values on n_bedroomss: 0
Number of missing values on n_bathrooms: 0


3. **price**

In [89]:
print("Number of missing values on price: " + str(df['price'].isna().sum()))


Number of missing values on price: 0


In [90]:
def hierarchical_median_impute(
    df, target_col, group_levels
):
    """
    df          : DataFrame
    target_col  : cột cần fill (price_per_m2)
    group_levels: list các list cột theo thứ tự ưu tiên
    """
    for group_cols in group_levels:
        median_map = (
            df
            .groupby(group_cols)[target_col]
            .median()
        )

        df[target_col] = df[target_col].fillna(
            df[group_cols].apply(
                lambda x: median_map.get(tuple(x), np.nan),
                axis=1
            )
        )
    return df

In [91]:
group_levels = [
    ["property_type", "district", "city_province"],
    ["property_type", "district"],
    ["property_type", "city_province"],
    ["property_type"]
]

df = hierarchical_median_impute(
    df,
    target_col = "price_per_m2",
    group_levels = group_levels
)

In [92]:
df.loc[
    df["price"].isna() & df["price_per_m2"].notna() & df["area"].notna(),
    "price"
] = df["price_per_m2"] * df["area"] / 1000   # triệu → tỷ

print("Number of missing values on price: " + str(df['price'].isna().sum()))

Number of missing values on price: 0


4. **front_road_width**

In [93]:
print("Number of missing values on front_road_width: " + str(df['front_road_width'].isna().sum()))

Number of missing values on front_road_width: 7984


In [94]:
# for Chung cư: front_road_width = 0
df.loc[
    (df["property_type"] == "Chung cư"),
    "front_road_width"
] = 0

print("Number of missing values on front_road_width: " + str(df['front_road_width'].isna().sum()))

Number of missing values on front_road_width: 4158


In [95]:
import numpy as np

def hierarchical_median_fill(df, target_col, group_levels):
    for group_cols in group_levels:
        median_map = (
            df
            .groupby(group_cols)[target_col]
            .median()
        )

        df[target_col] = df[target_col].fillna(
            df[group_cols].apply(
                lambda x: median_map.get(tuple(x), np.nan),
                axis=1
            )
        )
    return df

group_levels = [
    ["city_province", "district", "property_type"],
    ["district", "property_type"],
    ["property_type"]
]

df = hierarchical_median_fill(
    df,
    target_col = "front_road_width",
    group_levels = group_levels
)

global_median = df["front_road_width"].median()
df["front_road_width"] = df["front_road_width"].fillna(global_median)

print("Number of missing values on front_road_width: " + str(df['front_road_width'].isna().sum()))

Number of missing values on front_road_width: 0


5. **address**

In [104]:
df = df.dropna(subset = ['address', 'district', 'city_province'])
print("Number of missing values on address: " + str(df['address'].isna().sum()))
print("Number of missing values on district: " + str(df['district'].isna().sum()))
print("Number of missing values on city_province: " + str(df['city_province'].isna().sum()))

Number of missing values on address: 0
Number of missing values on district: 0
Number of missing values on city_province: 0


## Save preprocessed data

In [105]:
df.head(5)

,address,area,city_province,district,front_road_width,interior,latitude,legal,longitude,n_bathrooms,n_bedrooms,n_floors,price,property_type,source,has_interior_info,price_per_m2
0,"Dự án Masteri Thảo Điền, Đường Xa Lộ Hà Nội, P...",75.0,Hồ Chí Minh,Quận 2,0.00,Nội thất đầy đủ,10.802441,Sổ riêng,106.741950,2.0,2.0,2.0,7.7,Chung cư,batdongsancomvn,1,102.666667
1,"Phúc Lợi, Phúc Lợi, Long Biên, Hà Nội",72.0,Hà Nội,Long Biên,6.10,Nội thất đầy đủ,21.038832,Sổ riêng,105.934539,6.0,6.0,6.0,15.8,Nhà riêng,batdongsancomvn,1,219.444444
2,"Phường 12, Đà Lạt, Lâm Đồng",102.3,Lâm Đồng,Đà Lạt,5.00,Nội thất đầy đủ,11.973893,Sổ riêng,108.484595,1.0,2.0,2.0,7.5,Nhà riêng,batdongsancomvn,1,73.313783
3,"Đường Phạm Văn Chiêu, Phường 9, Gò Vấp, Hồ Chí...",60.9,Hồ Chí Minh,Gò Vấp,4.35,Nội thất cơ bản,10.849654,Sổ riêng,106.651222,2.0,2.0,1.0,6.6,Nhà riêng,batdongsancomvn,1,108.374384
4,"Đường Đồng Me, Phường Mễ Trì, Nam Từ Liêm, Hà Nội",148.0,Hà Nội,Nam Từ Liêm,6.50,Nội thất đầy đủ,21.011918,Sổ riêng,105.776275,41.0,41.0,10.0,59.0,Nhà riêng,batdongsancomvn,1,398.648649


In [106]:
print("Number of records: " + str(df.shape[0]))

Number of records: 20562


In [107]:
df.isna().sum()

address                  0
area                     0
city_province            0
district                 0
front_road_width         0
interior             11004
latitude                 0
legal                    0
longitude                0
n_bathrooms              0
n_bedrooms               0
n_floors                 0
price                    0
property_type            0
source                   0
has_interior_info        0
price_per_m2             0
dtype: int64

In [109]:
df.to_csv('../data/preprocessed/full_preprocessed.csv', index = False)